## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-18-23-49-00 +0000,wsj,Nike Sales Tick Up in Second Quarter; China We...,https://www.wsj.com/business/earnings/nike-sal...
1,2025-12-18-23-47-32 +0000,nypost,Pope Leo XIV’s oldest brother admitted to ‘eve...,https://nypost.com/2025/12/18/us-news/pope-leo...
2,2025-12-18-23-46-10 +0000,nypost,TikTok’s Chinese owner ByteDance inks deal to ...,https://nypost.com/2025/12/18/business/tiktoks...
3,2025-12-18-23-46-06 +0000,bbc,Ex-Nascar driver and his family among seven ki...,https://www.bbc.com/news/articles/c741r9ve3z7o...
4,2025-12-18-23-45-00 +0000,wsj,New York Mayor Adams Moves to Block Zohran Mam...,https://www.wsj.com/us-news/new-york-mayor-ada...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2388/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
74,trump,63
44,new,28
338,bondi,17
112,billion,14
138,year,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
327,2025-12-18-06-03-00 +0000,wsj,President Trump defended his handling of the e...,https://www.wsj.com/politics/policy/trump-defe...,119
14,2025-12-18-23-26-52 +0000,nypost,GOP senators push Trump to use $5 billion in R...,https://nypost.com/2025/12/18/world-news/gop-s...,119
102,2025-12-18-21-08-55 +0000,nyt,"Trump Media to Merge With TAE Technologies, a ...",https://www.nytimes.com/2025/12/18/business/tr...,117
373,2025-12-18-00-20-46 +0000,cbc,Trump trade rep targets Canada's beer and dair...,https://www.cbc.ca/news/politics/trump-cusma-c...,116
87,2025-12-18-21-37-00 +0000,wsj,Trump Media to Merge With Nuclear Fusion Firm ...,https://www.wsj.com/business/deals/trump-media...,114


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
327,119,2025-12-18-06-03-00 +0000,wsj,President Trump defended his handling of the e...,https://www.wsj.com/politics/policy/trump-defe...
107,96,2025-12-18-20-58-14 +0000,nypost,"New York Times columnist David Brooks, Google ...",https://nypost.com/2025/12/18/us-news/david-br...
104,63,2025-12-18-21-01-00 +0000,wsj,The Bank of Mexico lowered its benchmark inter...,https://www.wsj.com/articles/bank-of-mexico-ma...
85,63,2025-12-18-21-42-19 +0000,nypost,Muslim Bondi Beach hero who disarmed terrorist...,https://nypost.com/2025/12/18/world-news/musli...
14,56,2025-12-18-23-26-52 +0000,nypost,GOP senators push Trump to use $5 billion in R...,https://nypost.com/2025/12/18/world-news/gop-s...
61,47,2025-12-18-22-10-13 +0000,bbc,Kennedy Center to be renamed Trump-Kennedy Cen...,https://www.bbc.com/news/articles/cp84pxvp87eo...
42,46,2025-12-18-22-32-00 +0000,wsj,Former Nascar Driver Greg Biffle and Family Ki...,https://www.wsj.com/us-news/small-jet-crashes-...
51,45,2025-12-18-22-24-00 +0000,wsj,Economists warned a big drop in November infla...,https://www.wsj.com/economy/the-data-problems-...
16,45,2025-12-18-23-22-50 +0000,nypost,NYC City Council poised to pass NYPD body came...,https://nypost.com/2025/12/18/us-news/nyc-city...
348,43,2025-12-18-02-47-16 +0000,cbc,Another U.S. strike on alleged drug boat leave...,https://www.cbc.ca/news/world/us-military-east...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
